In [1]:
import torch
device  = "cuda:2" if torch.cuda.is_available() else "cpu"

from datasets import load_dataset

data_set = load_dataset("nlu_evaluation_data",split='train')
data_set = data_set.shuffle(seed=42)

labels = data_set.features["label"].names
num_labels = len(labels)

print(device)

Using custom data configuration default
Reusing dataset nlu_evaluation_data (/home/siddharthami_umass_edu/.cache/huggingface/datasets/nlu_evaluation_data/default/1.1.0/0416a5876d8240bd571f2bc2ad421cf6e6e88d938f8dcb5fd87b5af6033d6282)
Loading cached shuffled indices for dataset at /home/siddharthami_umass_edu/.cache/huggingface/datasets/nlu_evaluation_data/default/1.1.0/0416a5876d8240bd571f2bc2ad421cf6e6e88d938f8dcb5fd87b5af6033d6282/cache-ea2cd470d1f78acb.arrow


cuda:2


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
model = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=num_labels).to(device)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=64, truncation=True, return_tensors="pt")

data_set = data_set.map(tokenize_function)
data_set = data_set.rename_column("label", "labels")
data_set.set_format('torch', columns=["input_ids", "attention_mask", "labels"])
train_valid_test = data_set.train_test_split(test_size=0.3)
train_valid = train_valid_test['train'].train_test_split(test_size=0.3)

train_data =  train_valid['train']
valid_data = train_valid['test']
test_data = train_valid_test['test']


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'clas

In [3]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [4]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)
from transformers import get_scheduler

num_epochs = 20
num_training_steps = num_epochs * len(train_data)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32)
eval_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=32)


/home/siddharthami_umass_edu/miniconda3/envs/research/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [5]:
from datasets import load_metric

def evaluate():
    metric = load_metric("accuracy")
    model.eval()
    for batch in eval_dataloader:
        gbatch = {}
        for k, v in batch.items():
            if k in ["input_ids", "attention_mask"]:
                gbatch[k] = v[0].to(device)
            elif k == "labels":
                gbatch[k] = v.to(device) 

        
        with torch.no_grad():
            outputs = model(**gbatch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

    return metric.compute()

In [6]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

best_yet = 0.0
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        gbatch = {}
        for k, v in batch.items():
            if k in ["input_ids", "attention_mask"]:
                gbatch[k] = v[0].to(device)
            elif k == "labels":
                gbatch[k] = v.to(device) 
        outputs = model(**gbatch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    metrics = evaluate()
    print(f"epoch {epoch} metrics {metrics}")
    if metrics['accuracy'] > best_yet:
        test()
    

  0%|          | 0/252000 [00:00<?, ?it/s]

{'accuracy': 0.8927777777777778}


KeyboardInterrupt: 